In [3]:
import pandas as pd 
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV

# Read the dataset
df = pd.read_csv("train.csv")

# Drop irrelevant columns
columns_to_drop = ['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin']
df.drop(columns=columns_to_drop, inplace=True)

# Drop rows with missing values
df.dropna(inplace=True)

# Mapping for Embarked column
embarked_mapping = {'Q': 1, 'C': 2, 'S': 3}
df['Embarked'] = df['Embarked'].map(embarked_mapping)

# Assuming 'Survived' is the target column
X = df.drop(columns=['Survived'])
y = df['Survived']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define numerical and categorical features
numerical_features = ['Age']
categorical_features = ['Sex', 'Embarked']

# Create preprocessors for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder())
])

# Combine preprocessors using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a pipeline for preprocessing and SVM classification
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('svm', SVC())
])

# Define parameter grid for GridSearchCV
param_grid = {
    'svm__C': [0.1, 1, 10, 100],       # Test different values of C
    'svm__kernel': ['linear', 'rbf']   # Test different kernel functions
}

# Initialize GridSearchCV with pipeline and parameter grid
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')

# Perform grid search to find the best parameters
grid_search.fit(X_train, y_train)

# Get the best parameters and model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Train the best model
best_model.fit(X_train, y_train)

# Predict on the test set using the best model
y_pred_best = best_model.predict(X_test)

# Evaluate the best model
accuracy_best = accuracy_score(y_test, y_pred_best)
print("Best Model Accuracy:", accuracy_best)


Best Model Accuracy: 0.7762237762237763
